In [4]:
#!pip install --upgrade boto3

     |████████████████████████████████| 129 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 10.1 MB/s eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.18.16
    Uninstalling botocore-1.18.16:
      Successfully uninstalled botocore-1.18.16
  Attempting uninstall: boto3
    Found existing installation: boto3 1.15.16
    Uninstalling boto3-1.15.16:
      Successfully uninstalled boto3-1.15.16
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

awscli 1.18.157 requires botocore==1.18.16, but you'll have botocore 1.19.6 which is incompatible.


In [3]:
#!/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/bin/python -m pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 10.4 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [5]:
!pip show boto3

Name: boto3
Version: 1.16.6
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: UNKNOWN
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: sagemaker


In [1]:
import boto3
rekognition = boto3.client('rekognition')

In [2]:
import boto3

BUCKET = "image-classification-rekognition"
KEY = 'sadness/Alain_Cervantes_0001.jpg'
FEATURES_BLACKLIST = ("Landmarks", "Emotions", "Pose", "Quality", "BoundingBox", "Confidence")

def detect_faces(bucket, key, attributes=['ALL'], region="us-east-1"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
            "Bucket": bucket,
            "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response['FaceDetails']

for face in detect_faces(BUCKET, KEY):
    print("Face ({Confidence}%)".format(**face))
    # emotions
    for emotion in face['Emotions']:
        print("  {Type} : {Confidence}%".format(**emotion))
    # quality
    for quality, value in face['Quality'].items():
        print("  {quality} : {value}".format(quality=quality, value=value))


Face (99.99995422363281%)
  DISGUSTED : 57.83370590209961%
  CONFUSED : 36.49115753173828%
  SAD : 2.319786548614502%
  CALM : 1.4414823055267334%
  ANGRY : 1.152930736541748%
  SURPRISED : 0.37275615334510803%
  FEAR : 0.20832805335521698%
  HAPPY : 0.17985978722572327%
  Brightness : 78.3441390991211
  Sharpness : 4.3748369216918945


In [3]:
# Accuracy of the AWS Model (using Rekognition): approximately 60 percent

unique_emotions = ['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'contempt']
related_emotion = ['ANGRY', 'SURPRISED', 'DISGUSTED', 'FEAR', 'CALM', 'HAPPY', 'SAD', 'CONFUSED']

KEYS = []
s3_client = boto3.client('s3')
paginator = s3_client.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket='image-classification-rekognition',StartAfter='2018')
for page in result:
    if "Contents" in page:
        for key in page[ "Contents" ]:
            keyString = key[ "Key" ]
            KEYS.append(keyString)
print(len(KEYS))

def detect_faces(bucket, key, attributes=['ALL'], region="us-east-1"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
            "Bucket": bucket,
            "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response['FaceDetails']

correct = 0
for KEY in KEYS:
    print(KEY)
    for face in detect_faces(BUCKET, KEY):
        for emotion in face['Emotions']:
            real_emotion = unique_emotions.index(KEY.split('/')[0])
            predicted_emotion = related_emotion.index(emotion['Type'])
            if (real_emotion == predicted_emotion):
                correct += 1
            break
print(str(correct))

13681
anger/Al_Gore_0002.jpg
anger/Alejandro_Toledo_0004.jpg
anger/Alexander_Lukashenko_0001.jpg
anger/Alicia_Hollowell_0001.jpg
anger/Alvaro_Noboa_0002.jpg
anger/Anastasia_Myskina_0001.jpg
anger/Anastasia_Myskina_0003.jpg
anger/Angelina_Jolie_0005.jpg
anger/Arnold_Schwarzenegger_0023.jpg
anger/Atal_Bihari_Vajpayee_0010.jpg
anger/Ben_Braun_0001.jpg
anger/Bill_Graham_0005.jpg
anger/Billy_Crystal_0004.jpg
anger/Billy_Donovan_0001.jpg
anger/Bob_Huggins_0001.jpg
anger/Bob_Huggins_0004.jpg
anger/Brian_Scalabrine_0001.jpg
anger/Bud_Selig_0004.jpg
anger/Carlos_Arroyo_0001.jpg
anger/Carlos_Ortega_0001.jpg
anger/Celine_Dion_0008.jpg
anger/Charles_Moose_0001.jpg
anger/Charles_Moose_0002.jpg
anger/Charles_Moose_0004.jpg
anger/Charles_Moose_0006.jpg
anger/Charles_Moose_0013.jpg
anger/Colin_Powell_0007.jpg
anger/Colin_Powell_0030.jpg
anger/Colin_Powell_0043.jpg
anger/Colin_Powell_0055.jpg
anger/Colin_Powell_0149.jpg
anger/Colin_Powell_0180.jpg
anger/Colin_Powell_0181.jpg
anger/Colin_Powell_0188.jpg

disgust/Dwarakish_144.jpg
disgust/Dwarakish_145.jpg
disgust/Dwarakish_149.jpg
disgust/Dwarakish_15.jpg
disgust/Dwarakish_150.jpg
disgust/Dwarakish_151.jpg
disgust/Dwarakish_152.jpg
disgust/Dwarakish_153.jpg
disgust/Dwarakish_158.jpg
disgust/Dwarakish_16.jpg
disgust/Dwarakish_160.jpg
disgust/Dwarakish_162.jpg
disgust/Dwarakish_164.jpg
disgust/Dwarakish_165.jpg
disgust/Dwarakish_17.jpg
disgust/Dwarakish_170.jpg
disgust/Dwarakish_171.jpg
disgust/Dwarakish_173.jpg
disgust/Dwarakish_174.jpg
disgust/Dwarakish_175.jpg
disgust/Dwarakish_176.jpg
disgust/Dwarakish_177.jpg
disgust/Dwarakish_178.jpg
disgust/Dwarakish_179.jpg
disgust/Dwarakish_18.jpg
disgust/Dwarakish_180.jpg
disgust/Dwarakish_181.jpg
disgust/Dwarakish_183.jpg
disgust/Dwarakish_184.jpg
disgust/Dwarakish_185.jpg
disgust/Dwarakish_188.jpg
disgust/Dwarakish_189.jpg
disgust/Dwarakish_19.jpg
disgust/Dwarakish_190.jpg
disgust/Dwarakish_191.jpg
disgust/Dwarakish_198.jpg
disgust/Dwarakish_202.jpg
disgust/Dwarakish_203.jpg
disgust/Dwarakish

happiness/Alejandro_Toledo_0015.jpg
happiness/Alejandro_Toledo_0017.jpg
happiness/Alejandro_Toledo_0018.jpg
happiness/Alejandro_Toledo_0021.jpg
happiness/Alejandro_Toledo_0024.jpg
happiness/Alejandro_Toledo_0026.jpg
happiness/Alejandro_Toledo_0027.jpg
happiness/Alejandro_Toledo_0031.jpg
happiness/Alejandro_Toledo_0032.jpg
happiness/Alejandro_Toledo_0035.jpg
happiness/Alek_Wek_0001.jpg
happiness/Aleksander_Kwasniewski_0001.jpg
happiness/Aleksander_Kwasniewski_0002.jpg
happiness/Aleksander_Voloshin_0001.jpg
happiness/Alessandro_Nesta_0001.jpg
happiness/Alex_Barros_0001.jpg
happiness/Alex_Barros_0002.jpg
happiness/Alex_Cejka_0001.jpg
happiness/Alex_Ferguson_0001.jpg
happiness/Alex_Holmes_0001.jpg
happiness/Alex_Sink_0001.jpg
happiness/Alex_Sink_0002.jpg
happiness/Alex_Wallau_0001.jpg
happiness/Alex_Zanardi_0001.jpg
happiness/Alexander_Downer_0001.jpg
happiness/Alexander_Downer_0003.jpg
happiness/Alexander_Downer_0004.jpg
happiness/Alexandra_Pelosi_0001.jpg
happiness/Alexandra_Spann_0001.j

happiness/Ariel_Sharon_0064.jpg
happiness/Ariel_Sharon_0069.jpg
happiness/Ariel_Sharon_0074.jpg
happiness/Arif_Mardin_0001.jpg
happiness/Arminio_Fraga_0004.jpg
happiness/Arnold_Palmer_0001.jpg
happiness/Arnold_Schwarzenegger_0002.jpg
happiness/Arnold_Schwarzenegger_0003.jpg
happiness/Arnold_Schwarzenegger_0004.jpg
happiness/Arnold_Schwarzenegger_0005.jpg
happiness/Arnold_Schwarzenegger_0006.jpg
happiness/Arnold_Schwarzenegger_0007.jpg
happiness/Arnold_Schwarzenegger_0009.jpg
happiness/Arnold_Schwarzenegger_0010.jpg
happiness/Arnold_Schwarzenegger_0011.jpg
happiness/Arnold_Schwarzenegger_0012.jpg
happiness/Arnold_Schwarzenegger_0013.jpg
happiness/Arnold_Schwarzenegger_0014.jpg
happiness/Arnold_Schwarzenegger_0015.jpg
happiness/Arnold_Schwarzenegger_0018.jpg
happiness/Arnold_Schwarzenegger_0022.jpg
happiness/Arnold_Schwarzenegger_0027.jpg
happiness/Arnold_Schwarzenegger_0029.jpg
happiness/Arnold_Schwarzenegger_0030.jpg
happiness/Arnold_Schwarzenegger_0031.jpg
happiness/Arnold_Schwarzeneg

happiness/Brandon_Inge_0001.jpg
happiness/Brandon_Jones_0001.jpg
happiness/Branko_Crvenkovski_0001.jpg
happiness/Branko_Crvenkovski_0002.jpg
happiness/Brenda_Magana_0001.jpg
happiness/Brenda_Wilson_0001.jpg
happiness/Brendan_Fraser_0001.jpg
happiness/Brendan_Hansen_0001.jpg
happiness/Brendan_Hansen_0002.jpg
happiness/Brendan_Stai_0001.jpg
happiness/Brett_Hawke_0001.jpg
happiness/Brett_Hull_0001.jpg
happiness/Brian_Billick_0001.jpg
happiness/Brian_Campbell_0001.jpg
happiness/Brian_Cashman_0001.jpg
happiness/Brian_Cook_0001.jpg
happiness/Brian_De_Palma_0001.jpg
happiness/Brian_Grazier_0001.jpg
happiness/Brian_Griese_0001.jpg
happiness/Brian_Griese_0002.jpg
happiness/Brian_Heidik_0002.jpg
happiness/Brian_Henson_0001.jpg
happiness/Brian_Jordan_0001.jpg
happiness/Brian_Kerr_0001.jpg
happiness/Brian_Meadors_0001.jpg
happiness/Brian_Van_Dusen_0001.jpg
happiness/Brian_Wells_0001.jpg
happiness/Brian_Williams_0001.jpg
happiness/Bridget_Fonda_0001.jpg
happiness/Bridget_Fonda_0002.jpg
happiness/Br

...

neutral/Tim_Jones_0001.jpg
neutral/Tim_Lopes_0001.jpg
neutral/Tim_Pawlenty_0001.jpg
neutral/Tim_Robbins_0001.jpg
neutral/Tim_Robbins_0002.jpg
neutral/Tim_Robbins_0003.jpg
neutral/Tim_Salmon_0001.jpg
neutral/Tim_Welsh_0001.jpg
neutral/Timbul_Silaen_0001.jpg
neutral/Timothy_Coughlin_0001.jpg
neutral/Timothy_McVeigh_0001.jpg
neutral/Timothy_Rigas_0001.jpg
neutral/Tina_Pisnik_0001.jpg
neutral/Tirunesh_Dibaba_0001.jpg
neutral/Toby_Keith_0001.jpg
neutral/Tocker_Pudwill_0001.jpg
neutral/Todd_Haynes_0003.jpg
neutral/Todd_Haynes_0004.jpg
neutral/Todd_MacCulloch_0001.jpg
neutral/Todd_Parrott_0001.jpg
neutral/Todd_Petit_0001.jpg
neutral/Todd_Reid_0001.jpg
neutral/Todd_Robbins_0001.jpg
neutral/Todd_Wike_0001.jpg
neutral/Tom_Amstutz_0001.jpg
neutral/Tom_Christerson_0001.jpg
neutral/Tom_Coughlin_0001.jpg
neutral/Tom_Coverdale_0002.jpg
neutral/Tom_Craddick_0001.jpg
neutral/Tom_Craddick_0003.jpg
neutral/Tom_Crean_0001.jpg
neutral/Tom_Cruise_0004.jpg
neutral/Tom_Curley_0001.jpg
neutral/Tom_Daschle_0001

neutral/Vicente_Fox_0003.jpg
neutral/Vicente_Fox_0004.jpg
neutral/Vicente_Fox_0006.jpg
neutral/Vicente_Fox_0010.jpg
neutral/Vicente_Fox_0014.jpg
neutral/Vicente_Fox_0016.jpg
neutral/Vicente_Fox_0019.jpg
neutral/Vicente_Fox_0021.jpg
neutral/Vicente_Fox_0023.jpg
neutral/Vicente_Fox_0024.jpg
neutral/Vicente_Fox_0025.jpg
neutral/Vicente_Fox_0026.jpg
neutral/Vicente_Fox_0027.jpg
neutral/Vicente_Fox_0028.jpg
neutral/Vicente_Fox_0031.jpg
neutral/Vicente_Fox_0032.jpg
neutral/Victor_Hanescu_0001.jpg
neutral/Victoria_Beckham_0002.jpg
neutral/Victoria_Beckham_0003.jpg
neutral/Victoria_Clarke_0001.jpg
neutral/Victoria_Clarke_0002.jpg
neutral/Victoria_Clarke_0003.jpg
neutral/Victoria_Clarke_0004.jpg
neutral/Victoria_Clarke_0005.jpg
neutral/Vidar_Helgesen_0002.jpg
neutral/Vijay_Nambiar_0001.jpg
neutral/Viktor_Yushchenko_0001.jpg
neutral/Vin_Diesel_0001.jpg
neutral/Vin_Diesel_0002.jpg
neutral/Vince_Carter_0002.jpg
neutral/Vince_Carter_0003.jpg
neutral/Vince_Carter_0004.jpg
neutral/Vince_Dooley_0001.j

neutral/Zhang_Ziyi_0001.jpg
neutral/Zhong_Nanshan_0001.jpg
neutral/Zhu_Rongji_0002.jpg
neutral/Zhu_Rongji_0004.jpg
neutral/Zhu_Rongji_0006.jpg
neutral/Zhu_Rongji_0007.jpg
neutral/Zinedine_Zidane_0002.jpg
neutral/Zinedine_Zidane_0003.jpg
neutral/Zinedine_Zidane_0005.jpg
neutral/Zinedine_Zidane_0006.jpg
neutral/Zoran_Djindjic_0001.jpg
neutral/Zoran_Djindjic_0002.jpg
neutral/Zoran_Djindjic_0003.jpg
neutral/Zorica_Radovic_0001.jpg
neutral/Zulfiqar_Ahmed_0001.jpg
neutral/Zumrati_Juma_0001.jpg
neutral/Zydrunas_Ilgauskas_0001.jpg
sadness/Alain_Cervantes_0001.jpg
sadness/Alessandra_Cerna_0001.jpg
sadness/Alexander_Losyukov_0003.jpg
sadness/Amanda_Coetzer_0001.jpg
sadness/Amber_Frey_0001.jpg
sadness/Amy_Gale_0001.jpg
sadness/Andre_Agassi_0014.jpg
sadness/Andre_Agassi_0034.jpg
sadness/Andrew_Shutley_0001.jpg
sadness/Anette_Hosoi_0001.jpg
sadness/Anna_Jones_0001.jpg
sadness/Anna_Kournikova_0011.jpg
sadness/Antonio_Cassano_0001.jpg
sadness/Armando_Carrillo_0001.jpg
sadness/Arnaud_Lagardere_0001.jp

surprise/Alan_Greenspan_0003.jpg
surprise/Allen_Iverson_0001.jpg
surprise/Allison_Janney_0001.jpg
surprise/Alvaro_Silva_Calderon_0001.jpg
surprise/Alvaro_Uribe_0001.jpg
surprise/Alvaro_Uribe_0014.jpg
surprise/Amelie_Mauresmo_0002.jpg
surprise/Amelie_Mauresmo_0005.jpg
surprise/Amelie_Mauresmo_0009.jpg
surprise/Amelie_Mauresmo_0014.jpg
surprise/Anastasia_Myskina_0002.jpg
surprise/Anders_Fogh_Rasmussen_0001.jpg
surprise/Andre_Agassi_0012.jpg
surprise/Andreas_Vinciguerra_0001.jpg
surprise/Anna_Chicherova_0001.jpg
surprise/Anne_Donovan_0001.jpg
surprise/Antonio_Banderas_0004.jpg
surprise/Aram_Adler_0001.jpg
surprise/Arnold_Schwarzenegger_0024.jpg
surprise/Arnold_Schwarzenegger_0032.jpg
surprise/Aron_Ralston_0001.jpg
surprise/Arthur_Johnson_0001.jpg
surprise/Atal_Bihari_Vajpayee_0021.jpg
surprise/Augustin_Calleri_0004.jpg
surprise/Barry_Alvarez_0001.jpg
surprise/Begum_Khaleda_Zia_0002.jpg
surprise/Beyonce_Knowles_0001.jpg
surprise/Bill_Carmody_0001.jpg
surprise/Bill_Frist_0004.jpg
surprise/B

surprise/Monica_Seles_0001.jpg
surprise/Muffet_McGraw_0001.jpg
surprise/Mukesh_Ambani_0002.jpg
surprise/Nancy_Pelosi_0009.jpg
surprise/Nancy_Pelosi_0013.jpg
surprise/Naomi_Watts_0011.jpg
surprise/Natalie_Maines_0003.jpg
surprise/Natalie_Williams_0001.jpg
surprise/Nathalie_Gagnon_0001.jpg
surprise/Nathan_Lane_0001.jpg
surprise/Nicolas_Lapentti_0002.jpg
surprise/Nicolas_Macrozonaris_0001.jpg
surprise/Norah_Jones_0010.jpg
surprise/Oprah_Winfrey_0001.jpg
surprise/Oprah_Winfrey_0002.jpg
surprise/Orrin_Hatch_0002.jpg
surprise/Pablo_Khulental_0001.jpg
surprise/Paradorn_Srichaphan_0002.jpg
surprise/Paradorn_Srichaphan_0009.jpg
surprise/Pat_Summitt_0001.jpg
surprise/Patricia_Russo_0001.jpg
surprise/Patrick_Ewing_0001.jpg
surprise/Patti_Lank_0001.jpg
surprise/Paul_McNulty_0001.jpg
surprise/Pete_Carroll_0002.jpg
surprise/Pete_Carroll_0003.jpg
surprise/Phil_Mickelson_0001.jpg
surprise/Queen_Rania_0003.jpg
surprise/Raag_Singhal_0001.jpg
surprise/Raoul_Ruiz_0002.jpg
surprise/Reggie_Sanders_0001.jpg
